<a href="https://colab.research.google.com/github/Leonardo976/TTS/blob/main/TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Actualizar numpy y pandas a versiones compatibles
!pip install --upgrade numpy pandas

# Instalar otras bibliotecas necesarias
!pip install pyngrok gtts pyttsx3 torch torchvision torchaudio
!pip install scipy matplotlib
!apt-get install espeak






In [3]:
from pyngrok import ngrok

# Reemplaza "YOUR_AUTHTOKEN" con tu token de Ngrok
ngrok.set_auth_token("2mlWGQ5jURb6YApHndTBXBEA5ZC_7a4GtijtUonq7cnBLN7dQ")


In [ ]:
import pyttsx3

# Inicializar el motor
engine = pyttsx3.init()

# Listar voces disponibles
voices = engine.getProperty('voices')
available_voices = {}

# Filtrar y formatear las voces
for voice in voices:
    voice_name = voice.name  # Nombre completo de la voz
    voice_lang = voice.languages[0] if voice.languages else "unknown"  # Obtener el idioma
    country = voice_name.split(" ")[1] if len(voice_name.split(" ")) > 1 else "unknown"  # Tomar el país
    if voice.id not in available_voices:  # Evitar duplicados usando el ID
        available_voices[voice.id] = (voice_name, country, voice_lang)

# Mostrar las voces disponibles
for id, (name, country, lang) in available_voices.items():
    print(f'Voice: {name} - ID: {id} - Country: {country} - Language: {lang}')



In [5]:
# Iniciar un túnel en el puerto 5000
public_url = ngrok.connect(5000)
print("La URL pública de Ngrok es:", public_url)



La URL pública de Ngrok es: NgrokTunnel: "https://5b65-34-19-67-39.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from flask import Flask, request, jsonify, send_file, render_template_string
import librosa
import soundfile as sf
import numpy as np
import os
from gtts import gTTS  # Usar gTTS para TTS

# Crear aplicación Flask
app = Flask(__name__)

# Directorio temporal en Google Colab para guardar los archivos subidos
UPLOAD_FOLDER = '/content/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Función para aplicar fade in
def fade_in(audio, fade_duration):
    fade_in_samples = int(fade_duration * 44100)  # 44100 es la frecuencia de muestreo
    fade_in_curve = np.linspace(0, 1, fade_in_samples)
    audio[:fade_in_samples] *= fade_in_curve
    return audio

# Función para aplicar fade out
def fade_out(audio, fade_duration):
    fade_out_samples = int(fade_duration * 44100)
    fade_out_curve = np.linspace(1, 0, fade_out_samples)
    audio[-fade_out_samples:] *= fade_out_curve
    return audio

# En la parte de la vista principal de la API Flask
@app.route('/')
def index():
    return render_template_string('''
        <html>
        <head>
            <title>API de Procesamiento de Audio</title>
        </head>
        <body>
            <h1>API de Procesamiento de Audio</h1>
            <h2>1. Procesamiento de Audio</h2>
            <form action="/process_audio" method="post" enctype="multipart/form-data">
                <label for="file">Selecciona un archivo de audio (opcional):</label><br>
                <input type="file" name="file"><br><br>
                <label for="text_marks">Selecciona las opciones de audio:</label><br>
                <input type="checkbox" name="text_marks" value="slow"> Slow<br>
                <input type="checkbox" name="text_marks" value="fast"> Fast<br>
                <input type="checkbox" name="text_marks" value="pitch_up"> Pitch Up<br>
                <input type="checkbox" name="text_marks" value="pitch_down"> Pitch Down<br>
                <input type="checkbox" name="text_marks" value="reverse"> Reverse<br>
                <input type="checkbox" name="text_marks" value="fade_in"> Fade In<br>
                <input type="checkbox" name="text_marks" value="fade_out"> Fade Out<br>
                <input type="submit" value="Procesar Audio">
            </form>

            <h2>2. Texto a Voz</h2>
            <form action="/text_to_voice" method="post" enctype="multipart/form-data">
                <label for="text_to_speech">Texto para convertir a voz:</label><br>
                <textarea name="text_to_speech" rows="4" cols="50" placeholder="Escribe el texto aquí..."></textarea><br><br>
                <label for="voice_select">Selecciona la voz:</label><br>
                <select name="voice_select">
                    {% for id, (name, country, lang) in available_voices.items() %}
                    <option value="{{ id }}">{{ name }} ({{ country }}, {{ lang }})</option>
                    {% endfor %}
                </select><br><br>
                <input type="submit" value="Convertir a Voz">
            </form>
        </body>
        </html>
    ''', available_voices=available_voices)

# Ruta para procesar el audio
@app.route('/process_audio', methods=['POST'])
def process_audio():
    if 'file' not in request.files:
        return jsonify({"message": "No audio file uploaded."}), 400

    # Recibir el archivo de audio subido (opcional)
    audio_file = request.files.get('file')
    audio = None
    if audio_file:
        file_path = os.path.join(UPLOAD_FOLDER, audio_file.filename)
        audio_file.save(file_path)

        # Cargar el audio usando librosa
        audio, sr = librosa.load(file_path, sr=None)

    # Procesar marcas de texto
    text_marks = request.form.getlist('text_marks')  # Obtener una lista de opciones

    if audio is not None:
        for mark in text_marks:
            if mark == "slow":
                audio = librosa.effects.time_stretch(audio, rate=0.8)
            elif mark == "fast":
                audio = librosa.effects.time_stretch(audio, rate=1.2)
            elif mark == "pitch_up":
                audio = librosa.effects.pitch_shift(audio, n_steps=4, sr=sr)
            elif mark == "pitch_down":
                audio = librosa.effects.pitch_shift(audio, n_steps=-4, sr=sr)
            elif mark == "reverse":
                audio = audio[::-1]  # Revertir el audio
            elif mark == "fade_in":
                audio = fade_in(audio, fade_duration=1.0)  # Desvanecimiento de 1 segundo
            elif mark == "fade_out":
                audio = fade_out(audio, fade_duration=1.0)  # Desvanecimiento de 1 segundo

        # Guardar el archivo procesado
        output_path = os.path.join(UPLOAD_FOLDER, 'output.wav')
        sf.write(output_path, audio, sr)

        # Devolver el archivo procesado
        return send_file(output_path, as_attachment=True)
    else:
        return jsonify({"message": "No audio processing performed."}), 200

# Ruta para texto a voz
@app.route('/text_to_voice', methods=['POST'])
def text_to_voice():
    text_to_speech = request.form.get('text_to_speech', None)
    voice_select = request.form.get('voice_select', None)

    if not text_to_speech:
        return jsonify({"error": "Text must be provided."}), 400

    # Seleccionar la voz basada en la opción elegida
    selected_voice = available_voices.get(voice_select)
    engine.setProperty('voice', selected_voice)  # Establecer la voz seleccionada
    engine.setProperty('rate', 150)  # Ajustar velocidad de habla
    engine.setProperty('volume', 1.0)  # Ajustar volumen

    # Generar el audio de texto a voz
    audio_file_path = os.path.join(UPLOAD_FOLDER, 'text_to_voice_output.wav')
    tts = gTTS(text=text_to_speech, lang='es' if 'es' in selected_voice else 'en', slow=False)
    tts.save(audio_file_path)

    # Devolver el archivo procesado
    return send_file(audio_file_path, as_attachment=True)

# Iniciar la aplicación Flask
app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Sep/2024 22:03:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Sep/2024 22:03:16] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [30/Sep/2024 22:04:09] "POST /text_to_voice HTTP/1.1" 200 -


In [ ]:
from google.colab import files

# Subir el archivo de audio desde tu computadora
uploaded = files.upload()

# Listar los archivos subidos
for filename in uploaded.keys():
    print(f"Archivo subido: {filename}")



In [ ]:
import requests

# Usa la URL pública proporcionada por Ngrok
url = public_url + "/process_audio"  # Usa la URL de Ngrok

# Archivo de audio subido (cambia el nombre según corresponda)
files = {'file': open('/content/your_audio_file.wav', 'rb')}  # Cambia 'your_audio_file.wav' por el nombre del archivo subido
data = {'text_marks': 'pitch_up,slow'}

# Enviar solicitud POST a la API Flask
response = requests.post(url, files=files, data=data)

# Guardar el archivo de audio modificado devuelto por la API
with open("modified_audio.wav", 'wb') as f:
    f.write(response.content)

print("El audio modificado ha sido guardado como 'modified_audio.wav'.")
